Vagrantプロビジョニングで設定できない項目の初期設定

# 前処理

## インポート

In [ ]:
from fabric import Connection, Config
import subprocess, re, time

## 関数 - ping応答待ち(Windows)

In [ ]:
def wait_ping(ip, timeout=60):
    for _ in range(timeout):
        if subprocess.run(["ping", "-n", "1", "-w", "1000", ip], stdout=subprocess.DEVNULL).returncode == 0:
            print('OK')
            return
        time.sleep(1)
    raise TimeoutError(f"{ip} unreachable")

## 接続情報
IPアドレスはDefault Switchで割り当てられたDHCPアドレスを取得

In [ ]:
remote_user = 'vagrant'
host_name   = 'slurm-node1'
key_file    = f'.vagrant/machines/{host_name}/hyperv/private_key'
new_address = '192.168.128.11'

In [ ]:
result = subprocess.run(["vagrant", "ssh-config", host_name], capture_output=True, text=True)
match = re.search(r'^\s*HostName\s+(.+)$', result.stdout, re.MULTILINE)
remote_host = match.group(1)
print(remote_host)

## fabric設定

In [ ]:
config = Config(overrides={
    'run': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': False,'hide': False,'timeout': 20},
    'sudo': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': True,'hide': False,'timeout': 20,'password': None},
    'timeouts': {'connect': 5,'command': 30,},
})

# 接続

## SSH接続

In [ ]:
c = Connection(
    port=22,
    host=remote_host,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

# 処理実行

## Hyper-V内部ネットワークのIPアドレス設定
プロビジョニング完了のタイミングでNICを割り当てしているためこのタイミングで設定

In [ ]:
c.run('sudo ip a')

In [ ]:
c.sudo(f"""bash -c 'cat > /etc/netplan/99-hyperv-internal.yaml <<EOF
network:
  version: 2
  ethernets:
    eth1:
      dhcp4: false
      addresses:
        - {new_address}/24
EOF
'""")

In [ ]:
c.sudo("netplan apply", pty=False, warn=True, disown=True)

In [ ]:
wait_ping(new_address)

In [ ]:
c.close()
c = Connection(
    port=22,
    host=new_address,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

In [ ]:
c.run('sudo ip a')

## munge.key

In [ ]:
c.put('munge.key', '/home/vagrant/munge.key')

In [ ]:
c.sudo('cp /home/vagrant/munge.key /etc/munge/munge.key')

In [ ]:
c.sudo('chown munge:munge /etc/munge/munge.key')

In [ ]:
c.sudo('chmod 400 /etc/munge/munge.key')

In [ ]:
c.sudo('systemctl enable --now munge')

In [ ]:
c.sudo('systemctl status munge --no-pager')

## slurm-masterのSSH鍵を登録

In [ ]:
pubkey = open("id_rsa.pub").read().strip()

In [ ]:
c.run(f'echo "{pubkey}" >> ~/.ssh/authorized_keys')

# 切断

In [ ]:
c.close()